Importing section

In [98]:
import os
import string
from nltk import tokenize
from gensim.models import Word2Vec
from nltk import word_tokenize
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import pandas as pd

Reading the data from the folder and cleaning it.

In [99]:
sentences = []
for filename in os.listdir(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2"):
   with open(os.path.join(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\TelevisonMergedT1+T2", filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           s = s.lower()
           s = s.translate(str.maketrans('', '', string.punctuation))
           sentences.append(s)

tokensSentenceslist = []

for s  in sentences:
    wordsList = word_tokenize(s)
    tokensSentenceslist.append(wordsList)


##################### Uncomment below section for testing #########################
# print(len(sentences))
#
# for s in sentences:
#      print("The sentence is : ")
#      print(s)
#      print("-----------------------End of the sentence -------------")
#
# print (sentences)


# print (len(tokensSentenceslist))
# print (tokensSentenceslist)


In [100]:
print (len(sentences))
print(sentences[0:5])

38715
['advertising in television is becoming more and more clever', 'lots of those ads invade our subconsciousness and try to make us buy the product even if we do not actually need it', 'when it comes to children lots of them dont have enough money to afford these products', 'they dont even understand the concept of money', 'even if they wanted it they can not get the thing they were told they need']


Generating the Word2Vec Model

In [101]:
#model = Word2Vec(tokensSentenceslist, min_count=1)

#model = Word2Vec(tokensSentenceslist, vector_size=50, min_count=1, sg=1)
#model = Word2Vec(sentences=tokensSentenceslist, vector_size=100, workers=1, seed=42)

model = Word2Vec(window=10, min_count=2,workers=6,vector_size=50,seed=42,sg=0)
model.build_vocab(tokensSentenceslist, progress_per=1000)
model.train(tokensSentenceslist, total_examples=model.corpus_count, epochs=model.epochs)


##################### Uncomment below section for testing #########################


# print(list(model.wv.index_to_key))
# print(len(list(model.wv.index_to_key)))

(2373525, 3636460)

In [63]:
model.corpus_count
#model.epochs

38715

In [102]:
model.wv.most_similar("television")

[('televison', 0.936676561832428),
 ('tv', 0.8420758843421936),
 ('televsion', 0.5288699269294739),
 ('sholdnt', 0.49897295236587524),
 ('telvesion', 0.4982863664627075),
 ('certanly', 0.47414037585258484),
 ('frog', 0.4550800025463104),
 ('needless', 0.439322292804718),
 ('televion', 0.43875852227211),
 ('excemple', 0.43604832887649536)]

In [103]:
model.wv.most_similar("television")

[('televison', 0.936676561832428),
 ('tv', 0.8420758843421936),
 ('televsion', 0.5288699269294739),
 ('sholdnt', 0.49897295236587524),
 ('telvesion', 0.4982863664627075),
 ('certanly', 0.47414037585258484),
 ('frog', 0.4550800025463104),
 ('needless', 0.439322292804718),
 ('televion', 0.43875852227211),
 ('excemple', 0.43604832887649536)]

In [104]:
model.wv.most_similar("argument")

[('aspect', 0.8733988404273987),
 ('point', 0.8479516506195068),
 ('reason', 0.794777512550354),
 ('arguments', 0.7330032587051392),
 ('pro', 0.7329370379447937),
 ('topic', 0.7114583849906921),
 ('fact', 0.7027332782745361),
 ('positiv', 0.6803781390190125),
 ('theme', 0.6697933077812195),
 ('against', 0.6697912216186523)]

In [74]:
model.wv.most_similar("argument")

[('aspect', 0.8631858229637146),
 ('point', 0.8416258692741394),
 ('reason', 0.8248558640480042),
 ('statement', 0.6963896155357361),
 ('fact', 0.6947038173675537),
 ('topic', 0.6936120390892029),
 ('against', 0.6935745477676392),
 ('pro', 0.6923171281814575),
 ('arguement', 0.6699721813201904),
 ('arguments', 0.6554367542266846)]

In [105]:
model.wv.similarity("tv","against")

-0.1692446

In [106]:
def vectorize(list_of_docs, model, strategy):
    """Generate vectors for list of documents using a Word Emx`bedding.

    Args:
        list_of_docs: List of documents.
        model: Gensim Word Embedding.
        strategy: Aggregation strategy ("average", or "min-max".)

    Raises:
        ValueError: If the strategy is other than "average" or "min-max".

    Returns:
        List of vectors.
    """
    features = []
    size_output = model.vector_size
    embedding_dict = model

    if strategy == "min-max":
        size_output *= 2

    if hasattr(model, "wv"):
        embedding_dict = model.wv

    for tokens in list_of_docs:
        zero_vector = np.zeros(size_output)
        vectors = []
        for token in tokens:
            if token in embedding_dict:
                try:
                    vectors.append(embedding_dict[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            if strategy == "min-max":
                min_vec = vectors.min(axis=0)
                max_vec = vectors.max(axis=0)
                features.append(np.concatenate((min_vec, max_vec)))
            elif strategy == "average":
                avg_vec = vectors.mean(axis=0)
                features.append(avg_vec)
            else:
                raise ValueError(f"Aggregation strategy {strategy} does not exist!")
        else:
            features.append(zero_vector)
    return features

In [107]:
vectorized_docs = vectorize(tokensSentenceslist, model=model, strategy="average")
len(vectorized_docs), len(vectorized_docs[0])

(38715, 50)

In [89]:
print(model.wv["argument"])
print("#######################################################")
print(vectorized_docs[0])

[ 1.0834987   1.0775832  -2.4190793   1.3539352   0.7693121  -0.01766455
 -0.53748876  2.1623745  -0.5207145   1.694928   -0.2794287   1.4977117
  0.3757558  -1.0392004  -1.1415775  -0.37531468  0.23239893  1.364053
  1.6795251   0.6714833   0.8824193   1.365521    1.8032991  -0.67293257
  1.0104337  -0.03604451  1.0017824  -0.5705173  -0.49769476  1.6430551
 -1.352539    1.2772088   0.03308525 -2.0387952  -2.4316406  -1.2589905
  0.95908076 -2.2138608   0.6789476  -1.0684443   0.03440151  0.9796872
  0.9881222  -1.3170159  -2.6712847  -1.2676587   1.5911027  -0.6523292
 -1.2096913   0.46780825]
#######################################################
[ 0.09301123  0.02892222  0.02476533  0.29560983 -0.7727718   0.23872662
  0.4515218  -0.06879666 -0.03741706 -0.3008877  -0.20542604  0.8528051
 -0.14720449 -0.2436079  -1.0440915  -0.17934558 -0.21951494 -0.6070896
  0.41599515  0.6486483  -0.45739534  0.38212645  0.3662845   0.4625144
  0.98150975 -0.35466102  0.38813674 -0.44521755 -1.

In [108]:
def mbkmeans_clusters(X, k, mb=500, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_


In [109]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=10, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": sentences,
    "tokens": [" ".join(text) for text in tokensSentenceslist],
    "cluster": cluster_labels
})

For n_clusters = 10
Silhouette coefficient: 0.07
Inertia:219602.97053755476
Silhouette values:
    Cluster 3: Size:662 | Avg:0.67 | Min:0.04 | Max: 0.74
    Cluster 8: Size:2596 | Avg:0.15 | Min:-0.09 | Max: 0.37
    Cluster 1: Size:6052 | Avg:0.10 | Min:0.02 | Max: 0.25
    Cluster 0: Size:3614 | Avg:0.09 | Min:-0.05 | Max: 0.29
    Cluster 6: Size:5933 | Avg:0.04 | Min:-0.06 | Max: 0.19
    Cluster 4: Size:5220 | Avg:0.04 | Min:-0.09 | Max: 0.23
    Cluster 9: Size:4678 | Avg:0.04 | Min:-0.08 | Max: 0.22
    Cluster 5: Size:5180 | Avg:0.03 | Min:-0.10 | Max: 0.21
    Cluster 7: Size:2625 | Avg:0.02 | Min:-0.11 | Max: 0.22
    Cluster 2: Size:2155 | Avg:-0.01 | Min:-0.21 | Max: 0.23


### Evaluate top terms of the cluster

In [117]:
print("Top terms per cluster (based on centroids):")
for i in range(10): # number of cluster k should be put here!!
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=3)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
Cluster 0: they want parents 
Cluster 1: actually also the 
Cluster 2: summarizing i but 
Cluster 3: haben minuten dafã¼r 
Cluster 4: also bad that 
Cluster 5: just sayed adverisement 
Cluster 6: they that really 
Cluster 7: also advertises programms 
Cluster 8: allowed diricted young 
Cluster 9: they just maybe 


### Retrieve a random sample of documents for a given cluster


In [127]:
for i,t in enumerate(df_clusters.query(f"cluster == {0}").sample(10).iterrows()):
    print(t[1]["text"])
    print("-------------")

and they will learn that you dont need the latest toy to have fun and be happy
-------------
what the boy then does is to ask the parents if he could have this toy
-------------
some parents let their children watch televison because they are noises by their kids and when they need silence they let them watsch on the tv
-------------
the children learn that they cant have everything they see
-------------
a good thing about tv advertising directed toward children is that parents get new ideas what they could buy their children as a present if they ask for
-------------
another argument is that the children in most cases cant buy the products anyway because they dont have any money so they have to ask their parents if they can get the product or not and if the parents say no the children maybe learns that it cant always have everything it wants to have
-------------
if companies didnt do that children would not have as much fun as they are having because they dont get the kind of toys t

In [135]:
df_clusters.shape

df_clusters.tokens[0]

'advertising in television is becoming more and more clever'

### Most representative clusters

In [139]:
test_cluster = 1
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:2]:
    print(sentences[d])
    print("-------------")

those people do not want their children to see this role model for society because it is not quiet reality but when you think of the stories the generations before had been listening every eveing in the bed those stories have not been quiet realistic and it might be good this way to give your children the immage of a better world a better future than we live in today and that is why those movies and tv shows for kids show us a different world that wants to encourage children to use their creativity and immagination but these movies have to be worth their production costs so that they have to show commercial
-------------
rich companis can do a lot of it and get more costumers so they get even richer while poor companis can just do a little and will only reach a few people and wont get a lot of money so they stay poor and often get bought by a rich company so that in the end we only heve one or two very rich companis who dominate the market and no other company can rise because the rich